In [1]:
# CÉLULA 1: NOVAS IMPORTAÇÕES
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings

# --- NOVAS FERRAMENTAS DE PIPELINE ---
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

warnings.filterwarnings('ignore') 
print("Bibliotecas (incluindo Pipeline) importadas com sucesso!")

Bibliotecas (incluindo Pipeline) importadas com sucesso!


In [2]:
# CÉLULA 2: CARREGAR, CRUZAR (MERGE) E DEFINIR O ALVO
# (Esta célula está 99% igual, apenas mudamos o nome do dataframe final)

try:
    df_math = pd.read_csv('student_math_clean.csv')
    df_por = pd.read_csv('student_portuguese_clean.csv')
    print(f"Dados carregados: {len(df_math)} alunos de Mat. e {len(df_por)} alunos de Por.")
except FileNotFoundError:
    print("ERRO: Verifique se 'student_math_clean.csv' e 'student_portuguese_clean.csv' estão na pasta.")

chaves_aluno = [
    'school', 'sex', 'age', 'address_type', 'family_size', 'parent_status',
    'mother_education', 'father_education', 'mother_job', 'father_job',
    'school_choice_reason', 'guardian', 'travel_time', 'study_time',
    'class_failures', 'school_support', 'family_support', 'extra_paid_classes',
    'activities', 'nursery_school', 'higher_ed', 'internet_access',
    'romantic_relationship', 'family_relationship', 'free_time', 'social',
    'weekday_alcohol', 'weekend_alcohol', 'health'
]

df_combinado = pd.merge(
    df_math, 
    df_por, 
    on=chaves_aluno, 
    suffixes=('_mat', '_por')
)
print(f"Cruzamento concluído. {len(df_combinado)} alunos encontrados em AMBAS as turmas.")

# Criar o Alvo (y)
nota_de_corte = 10
em_risco_mat = (df_combinado['final_grade_mat'] < nota_de_corte)
em_risco_por = (df_combinado['final_grade_por'] < nota_de_corte)
df_combinado['em_risco_geral'] = (em_risco_mat | em_risco_por).astype(int)

# Definir y (o alvo)
y = df_combinado['em_risco_geral']

# Definir X (as features) - Note que NÃO ESTAMOS a pré-processar o X
colunas_para_excluir = [
    'student_id_mat', 'student_id_por', 'final_grade_mat', 
    'final_grade_por', 'em_risco_geral'
]
X_raw = df_combinado.drop(columns=colunas_para_excluir) # X agora está "cru" (raw)

print("X (features crus) e y (alvo) definidos.")

Dados carregados: 395 alunos de Mat. e 649 alunos de Por.
Cruzamento concluído. 162 alunos encontrados em AMBAS as turmas.
X (features crus) e y (alvo) definidos.


In [3]:
# CÉLULA 3: DEFINIR O PRÉ-PROCESSADOR (O "TRUQUE")

# Vamos dizer ao ColumnTransformer o que fazer com cada tipo de coluna

# 1. Colunas Binárias (yes/no, M/F, Urban/Rural)
# (Usamos OrdinalEncoder, que transforma [yes, no] em [1, 0])
binary_features = [
    'school_support', 'family_support', 'extra_paid_classes', 'activities',
    'nursery_school', 'higher_ed', 'internet_access', 'romantic_relationship',
    'sex', 'address_type', 'family_size', 'parent_status'
]
binary_transformer = OrdinalEncoder(categories=[['no', 'yes']] * 8 + [['M', 'F'], ['Rural', 'Urban'], ['Less than or equal to 3', 'Greater than 3'], ['Apart', 'Living together']])

# 2. Colunas Categoriais Ordinais (onde a ordem importa)
# (Definimos a ordem exata para o OrdinalEncoder)
ordinal_features = ['mother_education', 'father_education', 'travel_time', 'study_time']
education_cats = ['none', 'primary education (4th grade)', '5th to 9th grade', 'secondary education', 'higher education']
travel_cats = ['<15 min.', '15 to 30 min.', '30 min. to 1 hour', '>1 hour']
study_cats = ['<2 hours', '2 to 5 hours', '5 to 10 hours', '>10 hours']

ordinal_transformer = OrdinalEncoder(categories=[education_cats, education_cats, travel_cats, study_cats])

# 3. Colunas Categoriais Nominais (onde a ordem NÃO importa)
# (Usamos OneHotEncoder, que cria os dummies automaticamente)
one_hot_features = ['mother_job', 'father_job', 'school_choice_reason', 'guardian', 'school']
one_hot_transformer = OneHotEncoder(handle_unknown='ignore', drop='first') # drop='first' previne a multicolinearidade

# 4. Colunas Numéricas (que já são números e não mudam)
# (O 'remainder="passthrough"' diz ao pipeline para não mexer nelas)
numeric_features = [
    'age', 'class_failures', 'family_relationship', 'free_time', 'social',
    'weekday_alcohol', 'weekend_alcohol', 'health', 
    'absences_mat', 'grade_1_mat', 'grade_2_mat',
    'absences_por', 'grade_1_por', 'grade_2_por'
]

# 5. Juntar tudo no ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('binary', binary_transformer, binary_features),
        ('ordinal', ordinal_transformer, ordinal_features),
        ('onehot', one_hot_transformer, one_hot_features)
    ],
    remainder='passthrough' # Deixa as colunas numéricas (numeric_features) em paz
)

print("Pré-processador (ColumnTransformer) criado com sucesso.")

Pré-processador (ColumnTransformer) criado com sucesso.


In [4]:
# CÉLULA 4: CRIAR O PIPELINE

# Agora, "empacotamos" o pré-processador e o modelo num único objeto

rf_model = RandomForestClassifier(
    n_estimators=150,        
    max_depth=10,            
    class_weight='balanced', 
    random_state=42
)

# O Pipeline:
# 1. 'preprocessor': Roda o ColumnTransformer que definimos
# 2. 'classifier': Roda o Random Forest
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', rf_model)
])

print("Pipeline (Processador + Modelo) criado com sucesso.")

Pipeline (Processador + Modelo) criado com sucesso.


In [5]:
# CÉLULA 5: DIVIDIR DADOS (TREINO E TESTE)

# Usamos o X_raw (dados crus) e o y
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y, 
    test_size=0.25,      
    random_state=42,     
    stratify=y           
)

print(f"Total de dados: {len(X_raw)}")
print(f"Dados de Treino (X_train): {len(X_train)}")
print(f"Dados de Teste (X_test): {len(X_test)}")

Total de dados: 162
Dados de Treino (X_train): 121
Dados de Teste (X_test): 41


In [6]:
# CÉLULA 6: TREINAR O PIPELINE (UM ÚNICO COMANDO)

print("Iniciando treinamento do Pipeline...")

# O 'fit' agora faz TUDO: pré-processa e treina.
pipeline.fit(X_train, y_train)

print("Treinamento concluído.")

Iniciando treinamento do Pipeline...
Treinamento concluído.


In [7]:
# CÉLULA 7: VALIDAR O PIPELINE

print("\nAvaliando o Pipeline com os dados de Teste (crus)...")
y_pred = pipeline.predict(X_test) # O 'predict' também pré-processa e prevê

precisao_geral = accuracy_score(y_test, y_pred)
print(f"\nAcurácia Geral: {precisao_geral * 100:.2f}%")

print("\nRelatório de Classificação:")
print("---------------------------------")
print(classification_report(y_test, y_pred, target_names=['Sucesso (0)', 'Em Risco Geral (1)']))


Avaliando o Pipeline com os dados de Teste (crus)...

Acurácia Geral: 90.24%

Relatório de Classificação:
---------------------------------
                    precision    recall  f1-score   support

       Sucesso (0)       0.90      0.96      0.93        28
Em Risco Geral (1)       0.91      0.77      0.83        13

          accuracy                           0.90        41
         macro avg       0.90      0.87      0.88        41
      weighted avg       0.90      0.90      0.90        41



In [8]:
# CÉLULA 8: EXPORTAR O PIPELINE (O NOVO "CÉREBRO")

caminho_exportacao = "../modelo_risco_aluno2.pkl" 

try:
    # Salva o objeto 'pipeline' inteiro
    joblib.dump(pipeline, caminho_exportacao) 
    print(f"\nPipeline salvo com sucesso em: {caminho_exportacao}")
    print("Este arquivo agora contém o PRÉ-PROCESSADOR e o MODELO.")
except Exception as e:
    print(f"Erro ao salvar o pipeline: {e}")


Pipeline salvo com sucesso em: ../modelo_risco_aluno2.pkl
Este arquivo agora contém o PRÉ-PROCESSADOR e o MODELO.
